In [139]:
import sys
import pandas as pd
from sqlalchemy import create_engine


def load_data(messages_filepath, categories_filepath):
    '''
    Load and merges the 2 given files and returns them as 1 pandas dataframe object

            Parameters:
                    messages_filepath (str): filepath to messsages csv
                    categories_filepath (str): filepath to categories csv

            Returns:
                    df_merged (object): loaded files merged into one dataframe
    '''
    
    df_messages = pd.read_csv("disaster_messages.csv")
    df_categories = pd.read_csv("disaster_categories.csv")
    
    df_merged = pd.merge(df_messages, df_categories, on="id")
    
    return df_merged


def clean_data(df):
    '''
    Cleans the merged dataframe from load_data()

            Parameters:
                    df (object): uncleanded data as dataframe 
                    

            Returns:
                    df_cleaned (object): cleaned data as dataframe
    '''
    
    #create a new dataframe with a column for each categorie
    df_categories = pd.DataFrame(df["categories"].str.split(';', expand=True).values,
                 columns=[df["categories"].str.split(';')[0]])

    #reset multilevel index by changing columns list so single level index
    df_categories.columns = df["categories"].str.split(';')[0]

    #only keeping 0 or 1 from values and 
    for column in df_categories.columns:
        df_categories[column] = df_categories[column].str[-1:]

    #correct column names
    temp_col_name_list = []
    for col_name in df_categories.columns:
        temp_col_name_list.append(col_name[:-2])

    df_categories.columns = temp_col_name_list
    
    #cast all new columns to numeric as they are only 0 and 1
    df_categories[temp_col_name_list] = df_categories[temp_col_name_list].apply(pd.to_numeric)

    #concat both dataframes to get a single one with full informatio
    df = pd.concat([df, df_categories], axis=1)

    #drop categories and original column, because they are not needed for further analyis
    df_cleaned = df.drop(columns=["categories", "original"])

    return df_cleaned


def save_data(df, database_filename):
    '''
    Takes data as dataframe and stores it into a database at a given location

            Parameters:
                    df (object): data as dataframe 
                    database_filename (str): name of the database, where the df will be saved in
                    

            Returns:
                    None
    '''
    
    engine = create_engine('sqlite:///' + database_filename, echo=True)
    sqlite_connection = engine.connect()
    
    df.to_sql("disaster_respone", sqlite_connection, if_exists='replace')
    return  


In [140]:
messages_filepath, categories_filepath, database_filepath = "disaster_messages.csv", "disaster_categories.csv", "DisasterResponse.db"

print('Loading data...\n    MESSAGES: {}\n    CATEGORIES: {}'
      .format(messages_filepath, categories_filepath))
df = load_data(messages_filepath, categories_filepath)

print('Cleaning data...')
df = clean_data(df)

print('Saving data...\n    DATABASE: {}'.format(database_filepath))
save_data(df, database_filepath)

print('Cleaned data saved to database!')



Loading data...
    MESSAGES: disaster_messages.csv
    CATEGORIES: disaster_categories.csv
Cleaning data...
Saving data...
    DATABASE: DisasterResponse.db
2021-03-20 19:57:24,199 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-03-20 19:57:24,200 INFO sqlalchemy.engine.base.Engine ()
2021-03-20 19:57:24,201 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-03-20 19:57:24,202 INFO sqlalchemy.engine.base.Engine ()
2021-03-20 19:57:24,223 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("disaster_respone")
2021-03-20 19:57:24,224 INFO sqlalchemy.engine.base.Engine ()
2021-03-20 19:57:24,228 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("disaster_respone")
2021-03-20 19:57:24,229 INFO sqlalchemy.engine.base.Engine ()
2021-03-20 19:57:24,232 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-03-20 19:57:24,234 INFO sqlalchemy

In [132]:
df_messages = pd.read_csv("disaster_messages.csv")
df_categories = pd.read_csv("disaster_categories.csv")

In [133]:
df_categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [141]:
df.dtypes

id                         int64
message                   object
genre                     object
related                    int64
request                    int64
offer                      int64
aid_related                int64
medical_help               int64
medical_products           int64
search_and_rescue          int64
security                   int64
military                   int64
child_alone                int64
water                      int64
food                       int64
shelter                    int64
clothing                   int64
money                      int64
missing_people             int64
refugees                   int64
death                      int64
other_aid                  int64
infrastructure_related     int64
transport                  int64
buildings                  int64
electricity                int64
tools                      int64
hospitals                  int64
shops                      int64
aid_centers                int64
other_infr